In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

from glob import glob

import math
import random 

import numpy as np
from PIL import Image
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [10]:
import functools

def rsetattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

def rgetattr(obj, attr, *args):
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))

In [11]:
##################################################
# Model for Evaluation
##################################################

class ShufflePatchEvalNet(nn.Module):
  def __init__(self,aux_logits = False):

      super(ShufflePatchEvalNet, self).__init__()

      self.cnn = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64), 
        nn.ReLU(inplace=True),

        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64), 
        nn.ReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128), 
        nn.ReLU(inplace=True),

        nn.Conv2d(128, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128), 
        nn.ReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256), 
        nn.ReLU(inplace=True),

        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256), 
        nn.ReLU(inplace=True),

        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256), 
        nn.ReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512), 
        nn.ReLU(inplace=True),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512), 
        nn.ReLU(inplace=True),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512), 
        nn.ReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512), 
        nn.ReLU(inplace=True),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512), 
        nn.ReLU(inplace=True),

        nn.Conv2d(512, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512), 
        nn.ReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2)
      )
    
  def forward(self, patch):
    return self.cnn(patch)


# https://github.com/pratogab/batch-transforms/blob/master/batch_transforms.py

class ToTensor:
    """Applies the :class:`~torchvision.transforms.ToTensor` transform to a batch of images.
    """

    def __init__(self):
        self.max = 255
        
    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor of size (N, C, H, W) to be tensorized.
        Returns:
            Tensor: Tensorized Tensor.
        """
        return tensor.float().div_(self.max)


class Normalize:
    """Applies the :class:`~torchvision.transforms.Normalize` transform to a batch of images.
    .. note::
        This transform acts out of place by default, i.e., it does not mutate the input tensor.
    Args:
        mean (sequence): Sequence of means for each channel.
        std (sequence): Sequence of standard deviations for each channel.
        inplace(bool,optional): Bool to make this operation in-place.
        dtype (torch.dtype,optional): The data type of tensors to which the transform will be applied.
        device (torch.device,optional): The device of tensors to which the transform will be applied.
    """

    def __init__(self, mean, std, inplace=False, dtype=torch.float, device='cpu'):
        self.mean = torch.as_tensor(mean, dtype=dtype, device=device)[None, :, None, None]
        self.std = torch.as_tensor(std, dtype=dtype, device=device)[None, :, None, None]
        self.inplace = inplace
        
    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor of size (N, C, H, W) to be normalized.
        Returns:
            Tensor: Normalized Tensor.
        """
        if not self.inplace:
            tensor = tensor.clone()

        tensor.sub_(self.mean).div_(self.std)
        return tensor

    
class ShufflePatchFeatureExtractor():
    def __init__(self, weights_path):
        self.model = ShufflePatchEvalNet().to(device)
        
        print('Loading Weights...', weights_path)
        checkpoint = torch.load(weights_path, map_location=torch.device(device))
        self.model.load_state_dict(checkpoint['model_state_dict'])
        
        self.model.eval()
        
        self.transform_batch = transforms.Compose([
            ToTensor(),
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
        )
               
    # Numpy array of size (N, H, W, C)
    # Used for PIL images
    def evalRGB(self, patches):
        patches = torch.from_numpy(patches)
        patches = patches.permute(0, 3, 1, 2)
        patches = self.transform_batch(patches)
        output = self.model(patches.to(device))
        return output.cpu().detach().numpy()

    # Numpy array of size (N, H, W, C)
    # Used for CV2 images
    def evalBGR(self, patches):
        patches = patches[...,::-1].copy()
        return self.evalRGB(patches)
    

In [12]:
# Migrate Model

model = ShufflePatchEvalNet().to(device)

model_save_path = "/Users/racoon/Desktop/rotation_jigsaw_0820_0.0001_1.9327_43.59.pt"

print('Loading Checkpoint...', model_save_path)

checkpoint = torch.load(model_save_path, map_location=torch.device(device))

state_dict = checkpoint['model_state_dict']
print(state_dict.keys())

named_parameters = set(model.named_parameters())

with torch.no_grad():
    for k in state_dict.keys():
        if k in named_parameters: # k.startswith('cnn.'):
            print(k)
            rgetattr(model, k).copy_(state_dict[k])

model_save_path = "/Users/racoon/Desktop/rotation_jigsaw_migrated_0820_0.0001_1.9327_43.59.pt"

torch.save(
{
    'model_state_dict': model.state_dict(),
}, model_save_path)


Loading Checkpoint... /Users/racoon/Desktop/rotation_jigsaw_0820_0.0001_1.9327_43.59.pt
odict_keys(['cnn.0.weight', 'cnn.0.bias', 'cnn.1.weight', 'cnn.1.bias', 'cnn.1.running_mean', 'cnn.1.running_var', 'cnn.1.num_batches_tracked', 'cnn.3.weight', 'cnn.3.bias', 'cnn.4.weight', 'cnn.4.bias', 'cnn.4.running_mean', 'cnn.4.running_var', 'cnn.4.num_batches_tracked', 'cnn.7.weight', 'cnn.7.bias', 'cnn.8.weight', 'cnn.8.bias', 'cnn.8.running_mean', 'cnn.8.running_var', 'cnn.8.num_batches_tracked', 'cnn.10.weight', 'cnn.10.bias', 'cnn.11.weight', 'cnn.11.bias', 'cnn.11.running_mean', 'cnn.11.running_var', 'cnn.11.num_batches_tracked', 'cnn.14.weight', 'cnn.14.bias', 'cnn.15.weight', 'cnn.15.bias', 'cnn.15.running_mean', 'cnn.15.running_var', 'cnn.15.num_batches_tracked', 'cnn.17.weight', 'cnn.17.bias', 'cnn.18.weight', 'cnn.18.bias', 'cnn.18.running_mean', 'cnn.18.running_var', 'cnn.18.num_batches_tracked', 'cnn.20.weight', 'cnn.20.bias', 'cnn.21.weight', 'cnn.21.bias', 'cnn.21.running_mean', 

In [29]:
# Load Migrated Model
model_save_path = "/Users/racoon/Desktop/variation_2b_migrated_0135_0.001_1.4328_63.80.pt"
print('Loading Checkpoint...', model_save_path)
checkpoint = torch.load(model_save_path, map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])

Loading Checkpoint... /Users/racoon/Desktop/variation_2b_migrated_0135_0.001_1.4328_63.80.pt


NameError: name 'model' is not defined

In [30]:
patch_dim = 32
validation_image_paths = glob('/Users/racoon/Downloads/open-images-sample/*.jpg')
reuse_image_count = 0

class ShufflePatchDataset(Dataset):

  def __init__(self, image_paths, patch_dim, length, transform=None):
    self.image_paths = image_paths
    self.patch_dim = patch_dim
    self.length = length
    self.transform = transform
    self.image_reused = 0
    self.min_width = self.patch_dim
    
  def __len__(self):
    return self.length
  
  def __getitem__(self, index):
    # [y, x, chan], dtype=uint8, top_left is (0,0)
    
    image_index = int(math.floor((len(self.image_paths) * random.random())))
    
    if self.image_reused == 0:
      pil_image = Image.open(self.image_paths[image_index]).convert('RGB')
      self.pil_image = pil_image.resize((int(round(pil_image.size[0]/3)), int(round(pil_image.size[1]/3))))
      self.image_reused = reuse_image_count
    else:
      self.image_reused -= 1

    image = np.array(self.pil_image)

    # If image is too small, try another image
    if (image.shape[0] - self.min_width) <= 0 or (image.shape[1] - self.min_width) <= 0:
        return self.__getitem__(index)
    
    y_coord = int(math.floor((image.shape[0] - self.patch_dim) * random.random()))
    x_coord = int(math.floor((image.shape[1] - self.patch_dim) * random.random()))

    patch = image[y_coord:y_coord+self.patch_dim, x_coord:x_coord+self.patch_dim]

    if self.transform:
      patch = self.transform(patch)

    return patch


valdataset = ShufflePatchDataset(validation_image_paths, patch_dim, 1,
                         transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))

valloader = torch.utils.data.DataLoader(valdataset,
                                        batch_size=1,
                                        num_workers=0,
                                        shuffle=False)


In [31]:
feat.model.eval()

preprocess_input =  transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

data = next(iter(valloader))


img0 = data.to(device)
output = feat.model(img0)
output = output.cpu().detach().numpy()
print(output.shape)
output = output.reshape((512,))
print(output)

NameError: name 'model' is not defined